In [28]:
import wandb
import pickle
import tqdm

import numpy as np
import pandas as pd

import plotly.express as px

In [29]:
api = wandb.Api()

runs = api.runs('synapse_agent_experiments')

for run in runs:
    print(f'{run.name}: {run.state}')
    
# for run in tqdm.tqdm(runs):
#     run = api.run(run)
#     if run._step < 10:
#         print(f'Skipping {run.name} because it is too short {run._step} steps')
#         continue
#     print(f'{run.name}: {run.state}')

astral-snow-40: running
silver-glade-39: crashed
dutiful-darkness-38: failed
graceful-violet-36: crashed
revived-violet-21: crashed
misunderstood-brook-19: crashed
scarlet-lion-17: crashed
legendary-lake-16: crashed
lunar-field-15: killed
winter-haze-14: failed
upbeat-music-13: failed
stoic-sound-12: failed
noble-microwave-11: crashed
bumbling-music-10: failed
avid-yogurt-7: failed


In [30]:
run_path = 'opentensor-dev/synapse_agent_experiments/85sx63hh'
run = api.run(run_path)

df = pd.DataFrame([event for event in run.scan_history()])
df

,reference_length_words,challenge,reference,_step,model,desc,gpt_reference_time,challenge_eval_score,reference_from_challenge_eval_score,_runtime,...,goal,challenge_length_chars,reference_time,reference_length_chars,challenge_time,task_name,subtopic,topic,challenge_eval_response,_timestamp
0,284,How can I use the WebScopedWindowFocusAllowedI...,/*\n * Copyright (C) 2013 Google Inc. All righ...,0,HuggingFaceH4/zephyr-7b-beta,get help with debugging,29.740647,1,0.0,44.495710,...,ask for help fixing the broken piece of code,559,0.000011,2105,4.415413,debug,third_party/WebKit/Source/web/WebScopedWindowF...,js0701/chromium-crosswalk,<<1>>,1.704738e+09
1,284,How can I use the WebScopedWindowFocusAllowedI...,/*\n * Copyright (C) 2013 Google Inc. All righ...,1,HuggingFaceH4/zephyr-7b-beta,get help with debugging,33.191057,1,0.0,79.748364,...,ask for help fixing the broken piece of code,559,0.000003,2105,4.415413,debug,third_party/WebKit/Source/web/WebScopedWindowF...,js0701/chromium-crosswalk,<<1>>,1.704738e+09
2,284,How can I use the WebScopedWindowFocusAllowedI...,/*\n * Copyright (C) 2013 Google Inc. All righ...,2,HuggingFaceH4/zephyr-7b-beta,get help with debugging,20.211976,1,0.0,102.055537,...,ask for help fixing the broken piece of code,559,0.000003,2105,4.415413,debug,third_party/WebKit/Source/web/WebScopedWindowF...,js0701/chromium-crosswalk,<<1>>,1.704739e+09
3,284,How can I use the WebScopedWindowFocusAllowedI...,/*\n * Copyright (C) 2013 Google Inc. All righ...,3,HuggingFaceH4/zephyr-7b-beta,get help with debugging,29.559562,1,0.0,133.224231,...,ask for help fixing the broken piece of code,559,0.000003,2105,4.415413,debug,third_party/WebKit/Source/web/WebScopedWindowF...,js0701/chromium-crosswalk,<<1>>,1.704739e+09
4,284,How can I use the WebScopedWindowFocusAllowedI...,/*\n * Copyright (C) 2013 Google Inc. All righ...,4,HuggingFaceH4/zephyr-7b-beta,get help with debugging,27.410217,1,0.0,162.750339,...,ask for help fixing the broken piece of code,559,0.000003,2105,4.415413,debug,third_party/WebKit/Source/web/WebScopedWindowF...,js0701/chromium-crosswalk,<<1>>,1.704739e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,159,Can you provide a concise summary of the artis...,"Avi Mograbi, an Israeli documentary filmmaker ...",138,HuggingFaceH4/zephyr-7b-beta,get help with summarization,20.889604,1,1.0,4326.645135,...,summarize the following topic,156,7.947866,1089,1.115702,summarization,1956 births,Avi Mograbi,<<1>>,1.704743e+09
139,139,Can you provide a concise summary of the artis...,"Avi Mograbi, an Israeli documentary filmmaker ...",139,HuggingFaceH4/zephyr-7b-beta,get help with summarization,26.200194,1,1.0,4363.577437,...,summarize the following topic,156,7.269968,945,1.115702,summarization,1956 births,Avi Mograbi,<<1>>,1.704743e+09
140,131,How can I fix the broken code in the left side...,"<div id=""navigation"">\n <!-- <h1>目录111</h1> -...",140,HuggingFaceH4/zephyr-7b-beta,get help with debugging,22.750401,1,0.0,4390.754384,...,ask for help fixing the broken piece of code,252,0.000005,1462,1.860600,debug,_includes/lefttree.html,superlyao/blog,<<1>>,1.704743e+09
141,131,How can I fix the broken code in the left side...,"<div id=""navigation"">\n <!-- <h1>目录111</h1> -...",141,HuggingFaceH4/zephyr-7b-beta,get help with debugging,25.562541,1,0.0,4418.208039,...,ask for help fixing the broken piece of code,252,0.000003,1462,1.860600,debug,_includes/lefttree.html,superlyao/blog,<<1>>,1.704743e+09


In [31]:
df.task_name.value_counts()

task_name
summarization    90
qa               40
debug            13
Name: count, dtype: int64

In [ ]:
def load_results(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    
    df = pd.DataFrame(data)
    df.to_csv(path.replace('.pkl', '.csv'), index=False)
    return df


# df_gpt = load_results('gpt4-summarization.pkl')
# df_zep = load_results('zephyr-summarization.pkl')
# df_sol = load_results('solar-summarization.pkl').assign(model='solar')


solar_path = 'data/experiments_new_prototype_challenge_ref_evaluation_solar_results_1567_rows.csv'
zephyr_path = 'data/experiments_new_prototype_challenge_ref_evaluation_zephyr_results_878_rows.csv'


df_zep = pd.read_csv(zephyr_path)
df_sol = pd.read_csv(solar_path)

df = pd.concat([df_zep, df_sol], ignore_index=True) 


In [ ]:

df['challenge_tokens_per_second'] = df['challenge_length_words'] / df['challenge_time'] / 0.75
df['reference_tokens_per_second'] = df['reference_length_words'] / df['reference_time'] / 0.75
df

In [ ]:
# Analysis of generated text length
for c in ['challenge_length_words','reference_length_words']:
    px.histogram(df, x=c, color='model', 
                 opacity=0.5,
                 marginal='box', title=c,
                 width=800, height=600, template='plotly_white').show()


In [ ]:
# Analysis of generated text length
for c in ['challenge_tokens_per_second','reference_tokens_per_second']:
    px.histogram(df, x=c, color='model', 
                 opacity=0.5,
                 marginal='box', title=c,
                 width=800, height=600, template='plotly_white').show()

In [ ]:
# Analysis of evaluations generated text length
for c in ['challenge_time','reference_time']:
    px.histogram(df, x=c, color='model', 
                 opacity=0.5,
                 marginal='box', title=c,
                 width=800, height=600, template='plotly_white').show()

In [ ]:
# Analysis of evaluations generated text length
for c in ['challenge','reference']:
    px.scatter(df, x=f'{c}_length_words',  y=f'{c}_time', color='model',
                 opacity=0.3, title=c,
                 marginal_x='histogram', marginal_y='histogram',
                 width=800, height=600, template='plotly_white').show()

In [ ]:
score_cols = df.filter(like='score').columns.tolist()
score_cols

In [ ]:
df['roleplay_system_prompt_id'] = df['roleplay_system_prompt'].str.startswith('This is a roleplaying game where you are impersonating ').astype(int)
df['roleplay_system_prompt_label'] = df['roleplay_system_prompt_id'].map({0: 'Default', 1: 'Custom'})
df['roleplay_system_prompt_label'].value_counts()

In [ ]:
fig = px.histogram(df.loc[df.model!='solar', ['model', 'roleplay_system_prompt_label']+score_cols], facet_row='model', facet_col='roleplay_system_prompt_label',
             barmode = 'group', opacity = 0.7, log_y=False, 
             histnorm='probability',
             title = 'Distributions of Scores',
             width = 1200, height = 600, template='plotly_white')

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


In [ ]:
for col in score_cols:
    print(f'\n------- {col} --------')
    print(df.astype({col: float}).groupby('model')[col].value_counts())#.unstack().fillna(0)
    
    print(f'\n Fraction of {col} which are accepted by GPT4 judge')
    print(df.astype({col: float}).groupby('model')[col].mean())

In [ ]:
for idx, row in df.loc[df.model == 'zephyr','challenge'].items():
    print(row)
    break

In [ ]:
df_zep.iloc[0]

In [ ]:
print(df_zep.iloc[0].roleplay_system_prompt)

In [ ]:
print(df_zep.iloc[0].challenge)

In [ ]:
df_zep['challenge_contains_artefact'] = df_zep['challenge'].str.startswith(", such as ")

px.histogram(df_zep, x='challenge_eval_score', color='challenge_contains_artefact', barmode='group', opacity=0.5, title='Zephyr')

In [ ]:
df_zep

In [ ]:
df_zep.loc[df_zep['challenge_contains_artefact'] == False, 'challenge']

In [ ]:
df_zep['reference'].str[:15].value_counts()

In [ ]:
df_sol['reference'].str[:20].value_counts()

In [ ]:
for text in df_sol['challenge'].sample(10):
    print('-------------------------')
    print(text)
    print()